# We are moving from pure extraction keywords design to semantic retreival design
Embeddings extract semantic meaning, not exact words


**ScholarMind — Semantic Query Extraction Benchmark**



## Download and Prepare the SciFact dataset for testing

In [ ]:
# Imports
from beir.datasets.data_loader import GenericDataLoader
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from functools import lru_cache
import time

In [ ]:
!wget https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/scifact.zip -O scifact.zip
!unzip scifact.zip -d beir_data/

--2025-12-02 23:02:31--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/scifact.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2816079 (2.7M) [application/zip]
Saving to: ‘scifact.zip’

scifact.zip         100%[===================>]   2.69M  1.39MB/s    in 1.9s    

2025-12-02 23:02:33 (1.39 MB/s) - ‘scifact.zip’ saved [2816079/2816079]

Archive:  scifact.zip
   creating: beir_data/scifact/
   creating: beir_data/scifact/qrels/
  inflating: beir_data/scifact/qrels/train.tsv  
  inflating: beir_data/scifact/qrels/test.tsv  
  inflating: beir_data/scifact/corpus.jsonl  
  inflating: beir_data/scifact/queries.jsonl  


In [ ]:
# Load SciFact BEIR Dataset
data_path = "beir_data/scifact"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")

  0%|          | 0/5183 [00:00<?, ?it/s]

In [ ]:
# Use 1 sample query + its first positive document
sample_qid = list(queries.keys())[0]
sample_query = queries[sample_qid]

first_doc_id = list(corpus.keys())[0]
sample_doc = corpus[first_doc_id]["text"]

print("Sample Query:", sample_query)
print("Sample Document:", sample_doc[:120], "...")

Sample Query: 0-dimensional biomaterials show inductive properties.
Sample Document: Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development a ...


## Define and Benchmark Models

In [ ]:
# ✅ Corrected MODELS TO TEST
models = {
   "SciBERT": "allenai/scibert_scivocab_uncased",
    "SPECTER": "sentence-transformers/allenai-specter",
    "MPNet": "sentence-transformers/all-mpnet-base-v2",
    "MiniLM": "sentence-transformers/all-MiniLM-L6-v2"
}

results = []

for name, model_path in models.items():
    print(f"\n🔥 Benchmarking {name} ...")

    # Load SentenceTransformer model correctly
    model = SentenceTransformer(model_path)

    # ---- Query embedding ----
    start = time.time()
    q_vec = model.encode(sample_query, convert_to_numpy=True)
    latency_query = (time.time() - start) * 1000  # ms

    # ---- Document embedding ----
    d_vec = model.encode(sample_doc, convert_to_numpy=True)

    # ---- Metrics ----
    dim = len(q_vec)
    norm = np.linalg.norm(q_vec)
    sim = float(util.cos_sim(q_vec, d_vec))

    results.append({
        "Model": name,
        "Dim": dim,
        "Latency (ms)": round(latency_query, 2),
        "Vector Norm": round(norm, 2),
        "Query–Doc Similarity": round(sim, 4)
    })

# Display Results
df = pd.DataFrame(results)
df



🔥 Benchmarking SciBERT ...

🔥 Benchmarking SPECTER ...

🔥 Benchmarking MPNet ...

🔥 Benchmarking MiniLM ...


,Model,Dim,Latency (ms),Vector Norm,Query–Doc Similarity
0,SciBERT,768,129.99,17.600000,0.5054
1,SPECTER,768,383.98,21.959999,0.6242
2,MPNet,768,588.03,1.000000,0.1254
3,MiniLM,384,67.85,1.000000,0.1329


**Conclusion**
regarding to previous output that highlighting which model is most suitable for embedding queries in our project:

**SPECTER is the Best choice**
  - Highest semantic similarity → best for retrieving relevant scientific documents.

In [ ]:
# === So we Gonna use SPECTER  ===
best = {
    "model_name": "SPECTER",
    "model_path": "allenai/specter",
    "dim": 768  # typical embedding dimension for SPECTER
}

print("\n🏆 Using SPECTER Directly:")
print(f"• Name: {best['model_name']}")
print(f"• Path: {best['model_path']}")
print(f"• Dim: {best['dim']}")

# Load SPECTER
embedding_model = SentenceTransformer("sentence-transformers/allenai-specter")


🏆 Using SPECTER Directly:
• Name: SPECTER
• Path: allenai/specter
• Dim: 768


## Cached Query Embedding and Test Semantic Similarity

In [ ]:
# Cached query embedding to reduce repeated computation
@lru_cache(maxsize=200)
def extract_query_vector(query: str):
    """
    Generate embedding for a query using SPECTER with caching.
    Handles exceptions gracefully without fallback.
    """
    start = time.time()
    try:
        vector = embedding_model.encode(query).tolist()
        latency = round((time.time() - start) * 1000, 2)
        print(f"✅ Local Query Embedding Generated")
        print(f"• Latency: {latency} ms")
        return vector
    except Exception as e:
        print(f"⚠ Local model failed: {e}")
        return None  # Return None if embedding fails

In [ ]:
# Example queries
queries = [
    "CRISPR gene editing in humans",  # Direct
    "How scientists use CRISPR to modify human DNA for therapeutic purposes"  # Paraphrased
]

# Encode both queries
vectors = embedding_model.encode(queries, convert_to_numpy=True)
v1, v2 = vectors[0], vectors[1]

# Compute cosine similarity
sim_score = round(cosine_similarity([v1], [v2])[0][0], 4)

print("\n🔹 Semantic Similarity Score:")
print("•", sim_score)

if sim_score > 0.6:
    print("The queries share the same semantic intent!")
else:
    print("⚠ Semantic intent is weak — consider tuning models")



🔹 Semantic Similarity Score:
• 0.8312
The queries share the same semantic intent!
